# Bronze: Customers
Batch ingestion of customer data from CSV files into a Delta table.

In [0]:
dbutils.widgets.text("catalog", "", "Catalog")
dbutils.widgets.text("schema", "bronze", "Schema")
dbutils.widgets.text("source_path", "", "Source Path")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
source_path = dbutils.widgets.get("source_path")

target_table = f"{catalog}.{schema}.bronze_customers"

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name

full_source_path = source_path + "/customers/customers.csv"

df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(full_source_path)
    .withColumn("_load_ts", current_timestamp())
)

df.write.format("delta").mode("overwrite").saveAsTable(target_table)

row_count = spark.table(target_table).count()
display(spark.createDataFrame([(row_count, target_table)], ["row_count", "target_table"]))

In [0]:
import json
dbutils.notebook.exit(json.dumps({"status": "SUCCESS", "table": target_table, "rows": row_count}))